<center><b>  <span style="font-size:24px;">FINA 4335 -Final Project-<span style="color:red;">Part II <span></span></b>
<center><span style="font-size:22px;"><b>Due date: 12/11/2024 11:59 PM ET</b></span></center>
            
##### First name and Last name (team member 1): Stephen Magliocchino
##### First name and Last name (team member 2): Benjamin Damon
##### First name and Last name (team member 3): Benjamin Inglesby

## Instructions
1. Please rename the notebook. It should have the following format: **Final Project_Part II_Lastname1_Lastname2** if there are 2 members in the team.
1. Use this **template** and you are free to add more code/markdown cells. Change the markdown cells if necessary. Well format this notebook file.
1. After you answer all questions:
    1. Restart your kernel and run all cells
    1. Save your notebook and export your notebook as HTML
    1. Submit the **notebook** and **HTML** to Canvas, under Final Group Project.

## Some Grading Guidelines
1. All code cells (excluding evaluation cells) should be **executable**. 
    - Non-executable code cells will receive up to **50%** of grade deduction. To receive more partial credit for a non-executable code cell, ensure it includes sufficient comments.
1.  <span style="color: blue;">I grade on both **correctness** and **effort**</span>
    - **Suppose you don’t know how to generate a specific variable, then you <span style="color: red;">may use random values (with somewhat reasonable range) to replace the variable</span>. You will lose point for that variable but will get most credit for subsequent questions.**
1. Write efficient and clean code
    -  No repetitive code (An example of repetitive code: import the same csv file twice).
    - Informative variable names (if not specified in the project description).
    - Less than 2 uncommented `print`, `.describe()`, or similar quality check in every code cell or every 5 lines of code.
    - Every 2 occurrence of inefficient or inclean code will receive **1 point deduction**
1. Preserve as many observations as possible
    - Losing observations is considered partially incorrect code
    - Do not fill NAs
1. Include sufficient comments (at least 1 comment every 3 lines of code)
      - Failure to include sufficient comments will result in a **5 points** grade deduction
1. Format the markdown file such as including headings and brief question description
1. Use the methods and formulas **covered in the lecture notebooks**. Using formulas inconsistent to the lecture notebooks receives 20% of grade penalty
1. Reasonable code execution time
   - The code execution time for this Part should be less than 20 minutes, otherwise **deduct 1 point for every 5 minutes more code execuation time**
1. **10% of grade penalty per day late, receive 0% if the project is submitted after Dec 13 11:59 PM ET.**

In [32]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import scipy.stats as stats 
import statsmodels.api as sm
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline
import time
%config InlineBackend.figure_format = 'retina'
# Measure execution time
start_time_part2 = time.time()  # Start time

## Data preparation (5 points)
**Explanation of variables in 'weekly_rets.csv'**

- **DATE**: This variable represents the last trading day of each week.
- **PERMNO**: This is a unique identifier assigned to each security.
- **ret**: the stock's weekly return, expressed as a decimal.
  
**Explanation of variables in 'monthly_rets_funda.csv'**

- **DATE**: This variable represents the last trading day of each month.
- **PERMNO**: This is a unique identifier assigned to each security.
- **PRC**: reflects the **unadjusted** close price of a stock on the last trading day of the month. This value is the raw closing price without adjustments for corporate actions such as stock splits, dividends, or other modifications.
- **RET**: the stock's monthly return, expressed as a decimal. 
- **SHROUT**: The **shares outstanding** (SHROUT) variable represents the number of shares (in thousands) of a company’s stock that are currently held by all shareholders.
- **revenues**:  This variable represents the total revenues for a specific fiscal year. The variable is already 'lagged'.

In [33]:
# Write your code in the provided code cell; you can add more cells (same applies below, no further repetition).
weekly_rets = pd.read_csv('weekly_rets.csv', parse_dates=['DATE'])
monthly_rets = pd.read_csv('monthly_rets_funda.csv', parse_dates=['DATE'])
weekly_rets.sort_values(by=['PERMNO', 'DATE'], inplace=True)
monthly_rets.sort_values(by=['PERMNO', 'DATE'], inplace=True)
mrets_with_signals = monthly_rets.copy()

In [54]:
# Evaluation cell (used for grading-I don't have a TA)
# you don't need to worry at all if you think certain valuation cell is wrong (same applies below, no further repetition)
print(weekly_rets.head())
print(monthly_rets.head())

        DATE  PERMNO       ret
0 2008-01-04   10001 -0.028264
1 2008-01-11   10001 -0.013150
2 2008-01-18   10001 -0.004682
3 2008-01-25   10001  0.002904
4 2008-02-01   10001  0.026923
        DATE  PERMNO        PRC       RET    SHROUT  revenues
0 2008-01-31   10001  13.986000 -0.006076  2872.125    84.278
1 2008-02-29   10001   9.490400  0.021682  4340.655    84.278
2 2008-03-31   10001   8.956035 -0.052463  4342.653    84.278
3 2008-04-30   10001   8.591400 -0.036661  4342.653    84.278
4 2008-05-30   10001  10.889100  0.271356  4342.653    84.278


## Constructing factors (33 points; 15+8+5+5)

### Create signals  (3*5=15 points)

#### price_to_sales

In [35]:
# revenue is already 'lagged'
mrets_with_signals['price_to_sales'] = mrets_with_signals['PRC'] / mrets_with_signals['revenues']

In [36]:
# some quality check such as .describe() and print head of key variables
print(mrets_with_signals.describe())
print(mrets_with_signals.isnull().sum())
print(mrets_with_signals.head())

                                DATE         PERMNO            PRC  \
count                         527919  527919.000000  525406.000000   
mean   2015-12-02 04:09:19.313076224   58495.690371      37.219174   
min              2008-01-31 00:00:00   10001.000000       0.016983   
25%              2011-09-30 00:00:00   18838.000000       5.154840   
50%              2015-10-30 00:00:00   76750.000000      17.032950   
75%              2020-01-31 00:00:00   87377.000000      41.308650   
max              2023-12-29 00:00:00   93436.000000    6993.449750   
std                              NaN   32011.164148     109.502017   

                 RET        SHROUT       revenues  price_to_sales  
count  525388.000000  5.279190e+05  505997.000000   503601.000000  
mean        0.009336  1.336051e+05    4137.744604        1.843115  
min        -0.971201  8.991000e+00   -1964.999000     -165.275735  
25%        -0.072681  1.954144e+04      91.016000        0.010764  
50%         0.002023  4.20619

#### ret_2to12m

In [37]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.
mrets_with_signals['ret_2to12m'] = (1 + mrets_with_signals['RET']).shift(2) * (1 + mrets_with_signals['RET']).shift(12) - 1

In [38]:
# some quality check such as .describe() and print head of key variables



#### ret_2w

In [39]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.
mrets_with_signals['ret_2w'] = (1 + mrets_with_signals['RET']).shift(1) * (1 + mrets_with_signals['RET']) - 1

In [40]:
# some quality check such as .describe() and print head of key variables


#### mcap_quarter

In [41]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.
mrets_with_signals['mcap_quarter'] = mrets_with_signals['PRC'] * mrets_with_signals['SHROUT']

In [42]:
# some quality check such as .describe() and print head of key variables
print(mrets_with_signals.describe())
print(mrets_with_signals.isnull().sum())
print(mrets_with_signals.head())

                                DATE         PERMNO            PRC  \
count                         527919  527919.000000  525406.000000   
mean   2015-12-02 04:09:19.313076224   58495.690371      37.219174   
min              2008-01-31 00:00:00   10001.000000       0.016983   
25%              2011-09-30 00:00:00   18838.000000       5.154840   
50%              2015-10-30 00:00:00   76750.000000      17.032950   
75%              2020-01-31 00:00:00   87377.000000      41.308650   
max              2023-12-29 00:00:00   93436.000000    6993.449750   
std                              NaN   32011.164148     109.502017   

                 RET        SHROUT       revenues  price_to_sales  \
count  525388.000000  5.279190e+05  505997.000000   503601.000000   
mean        0.009336  1.336051e+05    4137.744604        1.843115   
min        -0.971201  8.991000e+00   -1964.999000     -165.275735   
25%        -0.072681  1.954144e+04      91.016000        0.010764   
50%         0.002023  4.

#### beta

In [55]:
# Each signal is only worth 3 points. Suppose you don’t know how to generate a specific signal, then you may use random values 
# (with somewhat reasonable range) to replace the signal. You will lose 3 points but will get most credit for subsequent questions.
three_factors_day = web.get_data_famafrench('F-F_Research_Data_Factors_daily', start=mrets_with_signals['DATE'].min(), end=mrets_with_signals['DATE'].max())[0][['Mkt-RF','RF']].div(100)
ff_monthly = (1 + three_factors_day).resample('M').prod() - 1
merged_data = pd.merge(mrets_with_signals, ff_monthly, left_on='DATE', right_index=True, how='left')
merged_data['excess_ret'] = merged_data['RET'] - merged_data['RF']  # stock excess returns
merged_data['excess_mkt'] = merged_data['Mkt-RF']

/var/folders/1b/vm9vwcxd62s4cvv71njdc7140000gv/T/ipykernel_85279/785955661.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  three_factors_day = web.get_data_famafrench('F-F_Research_Data_Factors_daily', start=mrets_with_signals['DATE'].min(), end=mrets_with_signals['DATE'].max())[0][['Mkt-RF','RF']].div(100)


In [58]:
betas = []
window_size = 24  # 24 months for rolling window

# Loop through each stock and calculate the rolling beta using covariance and variance
for permno in merged_data['PERMNO'].unique():
    stock_data = merged_data[merged_data['PERMNO'] == permno]
    
    stock_betas = []
    for start in range(len(stock_data) - window_size + 1):
        end = start + window_size
        window_data = stock_data.iloc[start:end]
        
        cov = window_data['excess_ret'].cov(window_data['excess_mkt'])
        var = window_data['excess_mkt'].var()
        beta = cov / var if var != 0 else np.nan  # Avoid division by zero
        stock_betas.append(beta)

In [18]:
# some quality check such as .describe() and print head of key variables


In [19]:
#############################################################################################################################
# Evaluation cell (used for grading-I don't have a TA)
#############################################################################################################################
some_vars = ['PERMNO','RET']+['price_to_sales', 'ret_2to12m', 'ret_2w', 'mcap_quarter', 'beta']
print(mrets_with_signals[some_vars].head())
mrets_with_signals[some_vars].describe()

KeyError: "['ret_2to12m', 'ret_2w', 'mcap_quarter', 'beta'] not in index"

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
mrets_with_signals.head()

### Form time-series portfolios (8 points) 

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(value_df.head())
value_df.describe()

In [ ]:
# Evaluation cell (do not change; used for grading)
print(momentum_df.head())
momentum_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(reversal_df.head())
reversal_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(size_df.head())
size_df.describe()

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(beta_df.head())
beta_df.describe()

### plot (5 points)

### Save the 5 long-short factor returns in a DataFrame called `factors_df`. (Do not drop any missing values). (5 points)

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(factors_df.head())
factors_df.describe()

## Evaluate the factors you created (14 points; 4+4+6)

### Estimate Fama-French 5-factor model for the 5 factor portfolios (in `factors_df`). Report the R squares, regression coefficients, and their p values in a single DataFrame called `ff5_factors_reg`. 

### Create `factors_sumstat`. 

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
print(ff5_factors_reg)
print(factors_sumstat)

### Summarize at least three takeaways related to investments based on your results in Part II so far. Cite credible sources that help explain and support your findings. (6 points)

## Factor combination strategy (‘Mission Impossible’) (8 points)
-  You can get partial credit if you have reasonable code.

In [ ]:
# Evaluation cell (used for grading-I don't have a TA)
impossible_df

In [ ]:
end_time_part2 = time.time()  # End time
execution_time_part2 = end_time_part2 - start_time_part2
# Calculate the number of full minutes
minutes = int(execution_time_part2 // 60)
# Calculate the remaining seconds
seconds = int(execution_time_part2 % 60)  
print(f"Execution time for Part II of the project:{minutes} minutes and {seconds} seconds")